In [29]:
### Economic Development Capstone

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import hashlib
from sklearn.model_selection import train_test_split
import scipy.stats as stats
import statistics

%matplotlib inline

pd.options.display.float_format = '{:.2f}'.format #Stop displaying scientific notation

### Load Data:

In [30]:
df_suc = pd.read_csv("edp_closed.csv") #Closed projects
df_can = pd.read_csv("edp_cancelled.csv") #Cancelled projects
df_wip = pd.read_csv("edp_under_contract.csv") #Projects in progress

### Rename Columns

In [31]:
#Successful Projects
df_suc.rename({'Company Name': 'company_name', 'County': 'county', 'City':'city', 'Status': 'status', 'Program':'program', 'Primary Funding Agreement':'primary_funding',
              'Award Date':'award_date', 'Total Project Cost':'total_project_cost', 'Capital Investment':'company_funding',
              'Direct Assistance Awarded':'direct_assistance_awarded','Tax Benefits Awarded':'tax_benefits_awarded',
              'Projected Jobs - Created':'projected_jobs_created', 'Projected Jobs - Retained':'projected_jobs_retained', 'Projected Jobs - Other': 'p_jobs_other', 'Projected Jobs - Total': 'p_jobs_total',
              'Projected Qualifying Wage': 'p_qualwage', 'Contracted Jobs - Created':'cont_jobs_created',
              'Contracted Jobs - Retained':'cont_jobs_retained', 'Contracted Jobs - Other Created':'cont_jobs_othercreated',
              'Contracted Jobs - Total':'cont_jobs_total', 'Contracted Qualifying Wage':'cont_qualwage',
              'Verified Total Project Cost':'verified_total_project_cost', 'Project Completion Date':'project_completion_date',
              'Final Jobs - Created':'final_jobs_created', 'Final Jobs - Retained':'final_jobs_retained',
              'Final Jobs - Other Created':'final_jobs_other_created', 'Final Jobs - Total':'final_jobs_total',
              'Project Maintenance Date':'project_maintenance_date', 'City Coordinates':'city_coordinates'}, axis=1, inplace=True)

In [32]:
#Cancelled/Failured Projects
df_can.rename({'Company Name': 'company_name', 'County': 'county', 'City':'city', 'Status': 'status', 'Program':'program', 'Primary Funding Agreement':'primary_funding',
              'Award Date':'award_date', 'Total Project Cost':'total_project_cost', 'Capital Investment':'company_funding',
              'Direct Assistance Awarded':'direct_assistance_awarded','Tax Benefits Awarded':'tax_benefits_awarded',
              'Projected Jobs - Created':'projected_jobs_created', 'Projected Jobs - Retained':'projected_jobs_retained', 'Projected Jobs - Other': 'p_jobs_other', 'Projected Jobs - Total': 'p_jobs_total',
              'Projected Qualifying Wage': 'p_qualwage', 'Contracted Jobs - Created':'cont_jobs_created',
              'Contracted Jobs - Retained':'cont_jobs_retained', 'Contracted Jobs - Other Created':'cont_jobs_othercreated',
              'Contracted Jobs - Total':'cont_jobs_total', 'Contracted Qualifying Wage':'cont_qualwage',
              #'Verified Total Project Cost':'verified_total_project_cost', 'Project Completion Date':'project_completion_date',
              #'Final Jobs - Created':'final_jobs_created', 'Final Jobs - Retained':'final_jobs_retained',
              #'Final Jobs - Other Created':'final_jobs_other_created', 'Final Jobs - Total':'final_jobs_total',
              #'Project Maintenance Date':'project_maintenance_date',
            'City Coordinates':'city_coordinates'}, axis=1, inplace=True)

In [33]:
#In-Progress Projects
#Note contacted qualifying wage instead of contracted
df_wip.rename({'Company Name': 'company_name', 'County': 'county', 'City':'city', 'Status': 'status', 'Program':'program', 'Primary Funding Agreement':'primary_funding',
              'Award Date':'award_date', 'Total Project Cost':'total_project_cost', 'Capital Investment':'company_funding',
              'Direct Assistance Awarded':'direct_assistance_awarded','Tax Benefits Awarded':'tax_benefits_awarded',
              'Projected Jobs - Created':'projected_jobs_created', 'Projected Jobs - Retained':'projected_jobs_retained', 'Projected Jobs - Other': 'p_jobs_other',
              'Projected Jobs - Total': 'p_jobs_total',
              'Projected Qualifying Wage': 'p_qualwage', 'Contracted Jobs - Created':'cont_jobs_created',
              'Contracted Jobs - Retained':'cont_jobs_retained', 'Contracted Jobs - Other Created':'cont_jobs_othercreated',
              'Contracted Jobs - Total':'cont_jobs_total', 'Contacted Qualifying Wage':'cont_qualwage',
              'Verified Total Project Cost':'verified_total_project_cost', 'Project Completion Date':'project_completion_date',
              'Final Jobs - Created':'final_jobs_created', 'Final Jobs - Retained':'final_jobs_retained',
              'Final Jobs - Other Created':'final_jobs_other_created', 'Final Jobs - Total':'final_jobs_total',
              'Project Maintenance Date':'project_maintenance_date', 'City Coordinates':'city_coordinates',
              'Date Reported':'date_reported',
              'Project Completion Jobs - Created':'end_of_project_completion_jobs_created',
                'Project Completion Jobs - Retained':'end_of_project_completion_jobs_retained',
                'Project Completion Jobs - Other Created':'end_of_project_completion_jobs_others_created',
                'Project Completion Jobs - Total':'end_of_project_completion_jobs_total',
                'Reported Jobs - Created':'reported_jobs_created',
                'Reported Jobs - Retained':'reported_jobs_retained',
                'Reported Jobs - Other Created':'reported_jobs_others_created',
                'Reported Jobs - Total':'reported_jobs_total',}, axis=1, inplace=True)
#print(df_wip.columns)

In [34]:
# Adding new columns
df_suc['project_type'] = '2' #Success = 2
df_can['project_type'] = '0' #Cancelled = 0
df_wip['project_type'] = '1' #WIP = 1

In [35]:
#Create our df by merging the 3 together
df = pd.concat([df_suc, df_can, df_wip]).copy() #Merge the 3 dataframes together

In [36]:
pd.set_option('display.max_columns', None)
df.head(5)

,company,county,city,status,program,primary_funding,award_date,tot_project_cost,capital_investment,direct_assistance_awarded,tax_benefits_awarded,p_jobs_created,p_jobs_retained,p_jobs_other,p_jobs_total,p_qualwage,cont_jobs_created,cont_jobs_retained,cont_jobs_othercreated,cont_jobs_total,cont_qualwage,verified_total_project_cost,project_completion_date,final_jobs_created,final_jobs_retained,final_jobs_other_created,final_jobs_total,project_maintenance_date,city_coordinates,project_type,end_of_project_completion_jobs_created,end_of_project_completion_jobs_retained,end_of_project_completion_jobs_others_created,end_of_project_completion_jobs_total,reported_jobs_created,reported_jobs_retained,reported_jobs_others_created,reported_jobs_total,date_reported
0,"DHI Group Inc. (f/k/a Dice Career Solutions, I...",Polk,Urbandale,Closed Contracts - Successful,130% Component,11-130-015,11/18/2010,1750000,1750000,225000,0,16,154,0,170,23.20,16,90,0,106,23.20,5127664.00,11/30/2013,16.00,90.00,0.00,106.00,11/30/2015,POINT (-93.736522 41.630343),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Wacker Chemical Corporation,Monroe,Eddyville,Closed Contracts - Successful,130% Component,12-130-012,11/18/2011,7706860,7706860,150000,344047,3,13,0,16,21.36,3,13,0,16,21.36,7746970.00,11/30/2014,3.00,13.00,0.00,16.00,11/30/2016,POINT (-92.630188 41.158178),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Environmental Lubricants Manufacturing, Inc.",Grundy,Grundy Center,Closed Contracts - Full,Value-Added Agriculture Component,11-VAAC-013,05/19/2011,415000,415000,60000,0,0,0,0,0,0.00,0,0,0,0,0.00,0.00,06/30/2017,0.00,0.00,0.00,0.00,06/30/2019,POINT (-92.768301 42.361475),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,McKesson Corporation,Cerro Gordo,Clear Lake,Closed Contracts - Successful,High Quality Jobs Program,15-HQJDF-034,02/20/2015,65242000,65242000,170000,4200000,164,0,0,164,14.79,164,0,0,164,14.79,70955084.00,02/28/2018,164.00,0.00,0.00,164.00,02/28/2020,POINT (-93.378746 43.137357),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Peregrine Financial Group, Inc.",Black Hawk,Cedar Falls,Bankruptcy,Community Economic Betterment Account,07-CEBA-044,06/13/2007,13030000,11650000,350000,893063,41,0,12,53,21.38,0,1,89,90,21.38,0.00,06/30/2010,0.00,0.00,0.00,0.00,06/30/2012,POINT (-92.4534 42.537809),2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,"HMA Group Holdings, LLC",Dallas,Waukee,Maintenance Period,High Quality Jobs Program,17-HQJTC-004,07/22/2016,8160257,1877621,0,987132,17,0,0,17,26.72,17,0,0,17,26.72,9588802.00,07/31/2019,NaN,NaN,NaN,NaN,07/31/2021,POINT (-93.885632 41.611984),1,17.00,0.00,0.00,17.00,0.00,0.00,0.00,0.00,06/30/2020
256,"M. H. Eby, Inc. and TNL Development, LLC",Story,Story City,Performance Period,High Quality Jobs Program,19-HQJDF-003,09/21/2018,4675000,4675000,176000,323000,22,0,42,64,26.11,22,0,42,64,26.11,0.00,09/30/2021,NaN,NaN,NaN,NaN,09/30/2023,POINT (-93.591059 42.185863),1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,06/30/2020
257,"AWOL Outdoors, Inc. d/b/a CAMP365",Monroe,Albia,Performance Period,High Quality Jobs Program,19-HQJDF-004,01/25/2019,3546000,3546000,500000,214800,156,0,156,312,16.05,156,0,156,312,16.05,0.00,01/31/2022,NaN,NaN,NaN,NaN,01/31/2024,POINT (-92.807111 41.026219),1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,06/30/2020
258,"TrinityRail Maintenance Services, Inc.",Butler,Shell Rock,Performance Period,High Quality Jobs Program,19-HQJTC-024,05/17/2019,60000000,48950000,0,3495000,28,0,235,263,21.43,28,0,235,263,21.43,0.00,05/31/2022,NaN,NaN,NaN,NaN,05/31/2024,POINT (-92.583205 42.710068),1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,06/30/2020


In [11]:
#Drop columns we don't need
df.drop(columns=['company', 'verified_total_project_cost', 'project_completion_date', 'final_jobs_created',
                 'final_jobs_retained', 'final_jobs_other_created', 'final_jobs_total',
                 'project_maintenance_date', 'end_of_project_completion_jobs_created',
                 'end_of_project_completion_jobs_retained',
                'end_of_project_completion_jobs_others_created',
                'end_of_project_completion_jobs_total', 'reported_jobs_created',
                'reported_jobs_retained', 'reported_jobs_others_created', 'reported_jobs_total',
                'date_reported', 'status', 'award_date', 'project_completion_date', 'city_coordinates', 'city'], inplace=True)

In [12]:
# Make a copy of the df
X = df.copy()

#Drop our response column from the X df
X = X.drop(columns='project_type')

#Need this for labels later
X_labels = X

#Create our y/response df
y = df['project_type']

In [13]:
#One hot encode X
# create an object of the OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False)

X = ohe.fit_transform(X)

In [14]:
#Scale data

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [15]:
#Create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
#Remember to always work on the train set

In [16]:
#Create and fit model
from sklearn.tree import DecisionTreeClassifier

tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train, y_train)

DecisionTreeClassifier()

In [17]:
from sklearn.metrics import accuracy_score

#Make prediction and check accuracy score
preds = tree_clf.predict(X_test)
acc_score = accuracy_score(preds, y_test)
print('Accuracy=%s' % (acc_score))

Accuracy=0.46774193548387094


In [18]:
#Grid Search

from sklearn.model_selection import GridSearchCV

#Create dictionary
param_grid = {'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], 'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10], 'min_samples_split': [2,3,4,5, 6, 7, 8]}

#Create GridSearch
grid_search_cv = GridSearchCV(tree_clf, param_grid, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

#Print out best parameters
print(grid_search_cv.best_params_)

print("The best parameters are: ", "max_depth: 2, max_leaf_nodes: 2, min_samples_split: 2")

Fitting 3 folds for each of 693 candidates, totalling 2079 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


{'max_depth': 2, 'max_leaf_nodes': 2, 'min_samples_split': 2}
The best parameters are:  max_depth: 2, max_leaf_nodes: 2, min_samples_split: 2


[Parallel(n_jobs=1)]: Done 2079 out of 2079 | elapsed:  1.3min finished


In [1]:
from sklearn.tree import export_text

tree_clf2 = DecisionTreeClassifier(max_depth=6, max_leaf_nodes = 7, min_samples_split = 8)
tree_clf2.fit(X, y)
cols = ohe.get_feature_names(input_features=X_labels.columns)
print(export_text(tree_clf2))

NameError: name 'DecisionTreeClassifier' is not defined